#Modules

In [3]:
import os
import numpy as np
import pandas as pd
import h5py
import tensorflow as tf
import keras
LABLE_FILE = 'C:/Users/007303173/Documents/Independent_Study_Project/nsd_data/ppdata/subj01/behav/responses.tsv'
FMRI_DIR = 'C:/Users/007303173/Documents/Independent_Study_Project/nsd_data/GLM-fmri-data/subj01'
FMRI_DIR_STND = 'C:/Users/007303173/Documents/Independent_Study_Project/nsd_data/standardized-betas/subj01'
TRIAL_PER_SESS = 750
# !pip install line_profiler
# %load_ext line_profiler

Here we get the name of all files in the directory containgn the betas

In [ ]:
  # returns a list of all files in the directory to be standardized
def getDirFiles(FMRI_DIR):
     files = [f for f in os.listdir(FMRI_DIR) if 
              os.path.isfile(os.path.join(FMRI_DIR, f)) and
              f[-5:] == '.hdf5']
     files.sort()                          
     return files

This shows each scan session is comprised of 750 scans. Each scan is of size (83, 104, 81)

In [ ]:
files = getDirFiles(FMRI_DIR)
f = files[0]
print(f)
f = os.path.join(FMRI_DIR, f)
scan1=[]
with h5py.File(f, "r") as file:
    print(file.keys())
    print(file['betas'].shape)
    scan1 = np.array(file['betas'])
print(scan1.shape[0])

In [ ]:
print(scan1.max())
print(scan1.min())
print(scan1.mean(0)[59,27,63])
mean = np.mean(scan1,axis=0)
print(mean[59,27,63])


In [8]:
print('filtering labels...')
res = pd.read_csv(LABLE_FILE, sep='\t')
res = pd.DataFrame(res)
print(res)
res = res.loc[res['SESSION'] <= 37]
res = res.loc[res['ISOLD'] == 1]
res = res.loc[res['ISCORRECT'] > -1]
print(res)
res = res.filter(items = ['ISCORRECT'] )
res
# for index, row in res.iterrows():
#     print(index, row['ISCORRECT'])

filtering labels...
       SUBJECT  SESSION  RUN  TRIAL  73KID  10KID        TIME  ISOLD  \
0            1        1    1      1  46003    626    0.505082      0   
1            1        1    1      2  61883   5013    0.505128      0   
2            1        1    1      3    829   4850    0.505175      0   
3            1        1    1      4  67574   8823    0.505221      0   
4            1        1    1      5  16021   9538    0.505267      0   
...        ...      ...  ...    ...    ...    ...         ...    ...   
29995        1       40   12     58  13774   8984  262.629551      1   
29996        1       40   12     59  66768   6026  262.629597      1   
29997        1       40   12     60  53168   4841  262.629644      1   
29998        1       40   12     61   1944   7323  262.629690      1   
29999        1       40   12     62   5034   9972  262.629736      1   

       ISCORRECT           RT  CHANGEMIND  MEMORYRECENT  MEMORYFIRST  \
0            1.0   803.529781         0.0  

,ISCORRECT
45,1.0
68,1.0
90,1.0
107,1.0
121,1.0
...,...
27745,1.0
27746,1.0
27747,0.0
27748,1.0


#Standardize all betas in the directory

---



In [4]:
class StandardizeDirectory:
  def __init__(self, loading_dir, dumping_dir, response_file):
    self.ld = loading_dir
    self.dd = dumping_dir
    self.rs = response_file
    self.index = 0
    self.offset = TRIAL_PER_SESS
    self.fmri_files = self.getDirFiles()
    self.file_handlers ={}
    self.open_files()

  def __del__(self):
    for value in self.file_handlers.items():
      value.close()
  # returns a list of all files in the directory to be standardized
  def getDirFiles(self):
     files = [f for f in os.listdir(self.ld) if 
              os.path.isfile(os.path.join(self.ld, f)) and
              f[-5:] == '.hdf5']
     files.sort()                          
     return files               

  def open_files(self): 
    for file_name in self.fmri_files:
      path = os.path.join(FMRI_DIR, file_name)
      self.file_handlers[file_name] = h5py.File(path, 'r')

  # Input: path of the responses.tsv file
  # Output: dictionary of indexes as keys with a corresponding class lable
  # in put the sesh number to filter , pass the session as an argument 
  def get_labels(self):
      print('filtering labels...')
      res = pd.read_csv(self.rs, sep='\t')
      res = pd.DataFrame(res)
      res = res.loc[res['SESSION'] <= 37]
      res = res.loc[res['ISOLD'] == 1]
      res = res.loc[res['ISCORRECT'] > -1]
      res = res.filter(items = ['ISCORRECT'] )
      # image_index = []
      # labels = []
      # for index, row in res.iterrows():
      #   image_index.append(index)
      #   labels.append(row['ISCORRECT'])
      return (res)

  def dump(self):
    path = os.path.join(self.dd, FMRI_DIR_STND+"/standardized_betas.hdf5")
    responses = self.get_labels()
    with h5py.File(path, 'a') as betas_file:
      
      betas_file.create_dataset("labels", data = labels)
      betas = np.empty(shape = (0, 83, 104, 81))
     

      for session in self.file_handlers:
        print('standardizing betas from session '+ session)
        sesh = np.array(self.file_handlers[session]['betas'])
        mean = sesh.mean(0)
        std = sesh.std(0)
        std = np.where(std == 0, 1, std)
        standardized = (sesh-mean)/std
        for index, row in responses: # this iters through tthe list of ID whick is the index of imaghes from 0-29999
          if index >= self.offset:
            self.index = self.offset
            self.offset += TRIAL_PER_SESS
            break
          else:
            expanded = standardized[(index- self.index)]
            expanded = np.expand_dims(expanded, axis= 0)
            betas = np.concatenate((betas, expanded), axis = 0)
            responses = responses.drop(index)
            
          
      betas_file.create_dataset("betas", data = betas)  

In [5]:
st = StandardizeDirectory(FMRI_DIR, FMRI_DIR_STND, LABLE_FILE)
st.dump()

filtering labels...
standardizing betas from session betas_session01.hdf5
standardizing betas from session betas_session02.hdf5
standardizing betas from session betas_session03.hdf5


IndexError: index -1455 is out of bounds for axis 0 with size 750